In [1]:
%pip install -q sentence-transformers

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances
import hdbscan


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Geclustertes DataFrame laden
df = pd.read_excel("../../data/df_de_final_duplikate_markiert.xlsx", engine="openpyxl")


In [3]:
# Nur eine Frage pro Duplikat-Gruppe oder alle Einzel-Fragen
df_unique = df.drop_duplicates(subset=["Duplikat_Gruppe"], keep="first").copy()

print(f"Anzahl einzigartiger Fragen: {len(df_unique)}")


Anzahl einzigartiger Fragen: 1316


In [4]:
# Sentence-BERT Modell laden (multilingual, funktioniert gut auf Deutsch)
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# Fragetexte vorbereiten
texts = df_unique['Frage_Text'].tolist()

# Embeddings erzeugen
vectors = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)


Batches:   0%|          | 0/42 [00:00<?, ?it/s]

In [5]:
# Cosine Distances berechnen (BERT-Embeddings sind normalerweise gut normalisiert)
distance_matrix = cosine_distances(vectors).astype(np.float64)


In [6]:
# HDBSCAN Clustering
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=2,
    min_samples=1,
    metric="precomputed",
    cluster_selection_epsilon=0.1  # macht HDBSCAN etwas "weicher"
)
labels_hdbscan = clusterer.fit_predict(distance_matrix)

# Cluster-Labels in DataFrame speichern
df_unique["Cluster_HDBSCAN"] = labels_hdbscan


In [7]:
# Effektivität prüfen: Wie viele einzigartige Fragen wurden sinnvoll geclustert?
anzahl_einzigartige = len(df_unique)
anzahl_geclusterte_einzigartige = (df_unique["Cluster_HDBSCAN"] != -1).sum()
effektivitätsquote = anzahl_geclusterte_einzigartige / anzahl_einzigartige * 100

print(f"Anzahl einzigartiger Fragen: {anzahl_einzigartige}")
print(f"Anzahl durch HDBSCAN geclusterter einzigartiger Fragen: {anzahl_geclusterte_einzigartige}")
print(f"Effektivität des Clustering (in %): {effektivitätsquote:.2f}%")


Anzahl einzigartiger Fragen: 1316
Anzahl durch HDBSCAN geclusterter einzigartiger Fragen: 908
Effektivität des Clustering (in %): 69.00%


In [8]:
# Exportieren
df_unique.to_excel("../../data/df_de_final_clustered_bert.xlsx", index=False, engine="openpyxl")

print("Clustering mit BERT erfolgreich abgeschlossen und exportiert.")


Clustering mit BERT erfolgreich abgeschlossen und exportiert.


In [9]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill

# 1. BERT-Cluster-Ergebnis laden
df = pd.read_excel("../../data/df_de_final_clustered_bert.xlsx", engine="openpyxl")

# 2. Nach Cluster sortieren
df_sorted = df.sort_values(by=["Cluster_HDBSCAN", "Frage_Text"]).reset_index(drop=True)

# 3. Neues Workbook erstellen
wb = Workbook()
ws = wb.active
ws.title = "BERT-Cluster sortiert"

# 4. Header schreiben
ws.append(df_sorted.columns.tolist())

# 5. Farben definieren
cluster_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")  # hellblau für Cluster-Trennung

# 6. Cluster schreiben mit visueller Trennung
current_cluster = None

for idx, row in df_sorted.iterrows():
    cluster_id = row["Cluster_HDBSCAN"]
    
    # Wenn neues Cluster beginnt, füge eine Trennzeile ein
    if cluster_id != current_cluster:
        if current_cluster is not None:
            ws.append(["" for _ in range(len(df_sorted.columns))])  # Leere Zeile
            for cell in ws[ws.max_row]:
                cell.fill = cluster_fill
        
        current_cluster = cluster_id

    # Normale Datenzeile schreiben
    ws.append(row.tolist())

# 7. Speichern
output_path = "../../data/df_de_final_clustered_bert_sorted_with_separators.xlsx"
wb.save(output_path)

print(f"Datei erfolgreich gespeichert: {output_path}")


Datei erfolgreich gespeichert: ../../data/df_de_final_clustered_bert_sorted_with_separators.xlsx
